# ATSC-500 Assignment III (scale heights & TPW)

In [1]:
import numpy as np
import pandas as pd
import netCDF4 as nc
from glob import glob
from os.path import basename
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
% matplotlib inline

# Import data

In [2]:
csv_names = glob('_data/ATSC-500/std_soundings/*.csv')

sound_dict={}
for name in csv_names:
    print('importing '+name)
    sound_dict[basename(name[:-4])] = pd.read_csv(name)

importing _data/ATSC-500/std_soundings/midsummer.csv
importing _data/ATSC-500/std_soundings/midwinter.csv
importing _data/ATSC-500/std_soundings/subwinter.csv
importing _data/ATSC-500/std_soundings/tropics.csv
importing _data/ATSC-500/std_soundings/subsummer.csv


In [3]:
# print out column names
sound_dict['midsummer'].head()

,Unnamed: 0,z,press,temp,rmix,den,o3den
0,0,0.0,101300.0,294.0,0.01400,1.1910,6.000000e-08
1,1,1000.0,90200.0,290.0,0.00930,1.0800,6.000000e-08
2,2,2000.0,80200.0,285.0,0.00585,0.9757,6.000000e-08
3,3,3000.0,71000.0,279.0,0.00343,0.8846,6.200001e-08
4,4,4000.0,62800.0,273.0,0.00189,0.7998,6.400001e-08


# Assignment i - calculate pressure & density scale height

Reference: [hydrostatic balance notes](https://clouds.eos.ubc.ca/~phil/courses/atsc500/docs/hydro.pdf)

**Pressure scale height**

---------------

Apply *reference equ.2* into the definition of pressure scale height ($H_p$):

$$
H_p = - \left(\frac{d\log{p}}{dz}\right)^{-1} =  \frac{R_dT_v}{g}
$$

The average of $H_p$ in the lower atmosphere:

$$
\overline{H_p} = \frac{1}{z_1 - z_0}\int^{z_1}_{z_0}{H_p dz} = \frac{R_d}{(z_1 - z_0)g}\int^{z_1}_{z_0}{T_v dz}
$$

**Density scale height**

---------------

Density scale height ($H_{\rho}$) is defined as:

$$
H_\rho = -\left(\frac{d\log{\rho}}{dz}\right)^{-1}
$$


Apply log and z-derivative to the ideal gas law: 

$$
p = \rho R_dT_v \Rightarrow \frac{d\log{p}}{dz} = \frac{d\log{\rho}}{dz} + \frac{d\log{T_v}}{dz}
$$

Which brings the reltion between $H_p$ and $H_\rho$ 

$$
\frac{1}{H_\rho} = \frac{1}{H_p} + \frac{1}{T_v}\frac{dT_v}{dz} = \frac{1}{H_p} + \frac{1}{T_v}\Gamma
$$

In [4]:
def Tv_midpoint(T, RMIX):
    Tv = (1+0.608*RMIX)*T
    return (Tv[1:] + Tv[:-1])/2.

def Hp_ave(T, Z, RMIX, thres=1e4):
    g = 9.8
    Rd = 286.888
    flag = (Z <= thres)
    Tv = Tv_midpoint(T[flag], RMIX[flag])
    return Rd*(Tv*np.diff(Z[flag])).sum()/(Z[flag][-1]-Z[flag][0])/g

def Hr_ave(T, Z, RMIX, thres=1e4):
    g = 9.8
    Rd = 286.888
    flag = (Z <= thres)
    Tv = Tv_midpoint(T[flag], RMIX[flag])
    dz = np.diff(Z[flag])
    gamma = 1*np.diff((1+0.608*RMIX[flag])*T[flag])/dz
    Hr = 1/(1/(Rd*Tv/g) + (1/Tv)*gamma)
    return (Hr*dz).sum()/(Z[flag][-1]-Z[flag][0])

In [5]:
for area in sound_dict.keys():
    df = sound_dict[area]
    T = df['temp'].as_matrix()
    Z = df['z'].as_matrix()
    RMIX = df['rmix'].as_matrix()
    print(area+': Hp = {:.4f}m, Hr = {:.4f}m'.format(Hp_ave(T, Z, RMIX, thres=1e4), Hr_ave(T, Z, RMIX, thres=1e4)))

midsummer: Hp = 7815.2694m, Hr = 9525.8942m
midwinter: Hp = 7280.9117m, Hr = 8617.1956m
subwinter: Hp = 7006.3113m, Hr = 7999.8264m
tropics: Hp = 7915.6647m, Hr = 9833.4523m
subsummer: Hp = 7566.0687m, Hr = 9289.5719m


# Assignment ii - calculate total precipitable water

total precipitable water ($W$) is defined as:

$$
W = \int_0^{z_{top}} \rho_v dz
$$

Apply idea gas law on water vapor to the definition of absolute humidity ($\rho_v$):

$$
\rho_v = \frac{m(\mathrm{H_2O})}{V(\mathrm{air})} = \frac{eM(\mathrm{H_2O})}{RT}\frac{V(\mathrm{H_2O})}{V(\mathrm{air})}
$$

Where $R$ is the ideal gas law constant with pressure unit as Pa, $\rho_v$ has the unit of $g/m^3$ 

Since water vapor is equally distributed in the air:

$$
V(\mathrm{H_2O}) = V(\mathrm{air})
$$

Then

$$
\rho_v = \frac{eM(\mathrm{H_2O})}{RT} = \frac{18}{8.314}\frac{e}{T} = 2.1650\frac{e}{T}
$$

According to *Dalton's law of partial pressure*:

$$
e = \frac{\gamma}{\gamma+\epsilon}p \quad \epsilon=\frac{M(\mathrm{H_2O})}{M(\mathrm{air})}=0.622
$$

In [10]:
def rho_midpoint(RMIX, P, T):
    P_midpoint = (P[1:] + P[:-1])/2.
    T_midpoint = (T[1:] + T[:-1])/2.
    RMIX_midpoint = (RMIX[1:] + RMIX[:-1])/2.
    e = RMIX_midpoint/(RMIX_midpoint+0.622)*P_midpoint
    return 2.1650*e/T_midpoint

def TPW(RMIX, P, T, Z, thres=1e4):
    flag = (Z <= thres)
    rho = rho_midpoint(RMIX[flag], P[flag], T[flag])
    dz = np.diff(Z[flag])
    return (rho*dz).sum()/(Z[flag][-1]-Z[flag][0])
    

In [11]:
for area in sound_dict.keys():
    df = sound_dict[area]
    T = df['temp'].as_matrix()
    Z = df['z'].as_matrix()
    P = df['press'].as_matrix()
    RMIX = df['rmix'].as_matrix()
    print(area+': total precipitable water = {} g/m2'.format(TPW(RMIX, P, T, Z)))

midsummer: total precipitable water = 2.9537241007742856 g/m2
midwinter: total precipitable water = 0.9046481753742888 g/m2
subwinter: total precipitable water = 0.4658669610888579 g/m2
tropics: total precipitable water = 4.131246232351342 g/m2
subsummer: total precipitable water = 2.107768842955067 g/m2
